In [ ]:
import numpy as np
import pandas as pd

from skimage import io
from skimage.filters import threshold_otsu

import os
import napari
from tqdm import tqdm
import pickle
import xarray as xr

import morphotrack.flow
import morphotrack.points
import morphotrack.track
import morphotrack.image
import morphotrack.distance

In [ ]:
io_directory = 'path_to_directory'

# load blood vessel
vessel = io.imread(os.path.join(io_directory,'R01_ch561_10um.tif'))

# load nuclei
nuclei = io.imread(os.path.join(io_directory,'R02_ch488_10um.tif'))

# load track information from the previous step
track_ds = xr.open_dataset(os.path.join(io_directory,"track_alignment.nc")).load()

# load regression model, this will be used for flux calculation
reg = pickle.load(open(os.path.join(io_directory,'model_.pkl'), 'rb'))

## local flux map. quality metrics.
### quality metrics and filitering

In [ ]:
std = track_ds.Total_warped.sel(displacement='t2s',track=track_ds.Standard.attrs['standard_seed'])

# get cosine similarities
cs = np.apply_along_axis(morphotrack.track.cosine_similarity, 1, track_ds.Total_warped.sel(displacement='t2s'), std)
# get mutual information
mi = np.apply_along_axis(morphotrack.track.mutual_information, 1, track_ds.Total_warped.sel(displacement='t2s'), std)

# thresholding with otsu
cs_thresh = threshold_otsu(cs)
mi_thresh = threshold_otsu(mi)

# save in track_ds
qc_array = xr.DataArray(np.stack([cs,mi]).T,
                        coords={'track': track_ds.track,
                             'metrics': ['cs','mi']},
                        dims=['track', 'metrics'],
                        attrs=dict(cs_thresh=cs_thresh,mi_thresh=mi_thresh)
                        )

track_ds['Similarity'] = qc_array

### Get various properties

In [ ]:
# convert position to distance from surface
intervals = morphotrack.distance.position2distance_xr(track_ds.Position)
dist = intervals.fillna(0).cumsum(dim='time')

In [ ]:
# convert time to laminar position
laminar_position = []
for tr in track_ds.track.data:
    interp_position = track_ds.Position.sel(track=tr).interp(time=track_ds.Total_arg.sel(displacement='t2s').sel(track=tr).data)
    interp_position = interp_position.assign_coords(time=track_ds.time)
    laminar_position.append(interp_position)
laminar_position = xr.concat(laminar_position,dim='track')
laminar_position = laminar_position.rename(time='warped_time')

In [ ]:
# calculate local flux
radius = 10 # * 10 micrometer
get_norm_flow = morphotrack.flow.flow_to_normflow(reg)
local_flux = []
for t in tqdm(laminar_position.warped_time.data):
    local_flux.append(morphotrack.points.get_local_flux(laminar_position.sel(warped_time=t).data, get_norm_flow, radius))
local_flux = np.stack(local_flux)
local_flux = xr.DataArray(local_flux,
                          coords={'warped_time': laminar_position.warped_time,
                                  'track': laminar_position.track},
                          dims=['warped_time', 'track'],
                          attrs=dict(radius=radius)
                          )

In [ ]:
track_ds['Distance'] = dist
track_ds['Warped_position'] = laminar_position
track_ds['Local_flux'] = local_flux

In [ ]:
track_ds.to_netcdf(os.path.join(io_directory,"track_ds_profiles.nc"))